<a href="https://colab.research.google.com/github/Prabhakars367/handwritten_text_recogn./blob/main/DOC_Q_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import torch
!pip install transformers

In [64]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at gpt2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
!pip install PyPDF2
!pip install PyMuPDF
!pip install python-docx


In [66]:
import PyPDF2
import fitz  # PyMuPDF
import docx
import os

def extract_text_from_file(file_path):
    text = ""

    if file_path.lower().endswith('.pdf'):
        with open(file_path, 'rb') as pdf:
            pdf_reader = PyPDF2.PdfReader(pdf)
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()

    elif file_path.lower().endswith('.xps') or file_path.lower().endswith('.oxps') or file_path.lower().endswith('.cbz') or file_path.lower().endswith('.cbr') or file_path.lower().endswith('.cb7'):
        pdf_document = fitz.open(file_path)
        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)
            text += page.get_text("text")

    elif file_path.lower().endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as txt:
            text = txt.read()

    elif file_path.lower().endswith('.docx'):
        doc = docx.Document(file_path)
        for paragraph in doc.paragraphs:
            text += paragraph.text + '\n'

    else:
        print(f"Unsupported file type: {file_path}")

    return text

# Path for input
input_file_path = "/content/story1.txt"

document_text = extract_text_from_file(input_file_path)

# output path
output_file_path = "/content/output.txt"


with open(output_file_path, 'w', encoding='utf-8') as output_file:
    output_file.write(document_text)

print(f"Text extracted from {input_file_path} and saved to {output_file_path}.")


Text extracted from /content/story1.txt and saved to /content/output.txt.


In [67]:
!pip install pytesseract
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [ ]:
!pip install pytesseract pillow


In [68]:
import pytesseract
from PIL import Image
import cv2
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'
image_path = '/content/hand_img.jpeg'
img = cv2.imread(image_path)
text = pytesseract.image_to_string(img)
print("Recognized Text:")
print(text)
document_text=text


Recognized Text:
LAST Wedneschy
We had » good
Team building



In [69]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [70]:
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

def read_text(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin-1') as file:
            text = file.read()
    return text


def sanitize_text(text):
    sentences = nltk.sent_tokenize(text)
    word_freq = {}
    stop_words = set(stopwords.words('english'))

    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        for word in words:
            word = word.lower()
            if word not in stop_words:
                if word not in word_freq:
                    word_freq[word] = 1
                else:
                    word_freq[word] += 1

    max_freq = max(word_freq.values())
    for word in word_freq:
        word_freq[word] = (word_freq[word] / max_freq)

    return word_freq, sentences

def calculate_sentence_scores(sentences, word_freq):
    sentence_scores = {}

    for sentence in sentences:
        for word in nltk.word_tokenize(sentence):
            if word in word_freq:
                if sentence not in sentence_scores:
                    sentence_scores[sentence] = word_freq[word]
                else:
                    sentence_scores[sentence] += word_freq[word]

    return sentence_scores

def generate_summary(file_path, num_sentences=5):
    text = read_text(file_path)
    word_freq, sentences = sanitize_text(text)
    sentence_scores = calculate_sentence_scores(sentences, word_freq)

    ranked_sentences = sorted(
        ((sentence, score) for sentence, score in sentence_scores.items()),
        key=lambda x: x[1],
        reverse=True
    )

    summary_sentences = [sentence for sentence, score in ranked_sentences[:num_sentences]]
    summary = ' '.join(summary_sentences)
    return summary

# Replace
input_file = '/content/output.txt'
num_sentences = 5

summary = generate_summary(input_file, num_sentences)
print("Summary:")
print(summary)


Summary:
From then on, John made sure to keep his spare key in a safe place, so he wouldn’t have to worry about being locked out again. He went to his neighbor’s house and asked if he could borrow the key. He had lost the key to his house and he couldn’t get inside. Suddenly, he remembered that he had given a spare key to his neighbor. His neighbor happily gave him the key and John was able to get inside his house.


In [71]:
document = document_text
question = "What happened last wedneschy?"

In [72]:
inputs = tokenizer(question, document, return_tensors="pt")

In [73]:
inputs = tokenizer(question, document, return_tensors="pt", truncation=True, padding=True)
start_positions, end_positions = model(**inputs).values()

answer_start = torch.argmax(start_positions)
answer_end = torch.argmax(end_positions) + 1
answer = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end])

print("Question:", question)
print("Answer:", answer)

Question: What happened last wedneschy?
Answer: What happened last
